# **MODELAGEM E ESTIMATIVA DOS PARÂMETROS CINÉTICOS PARA PRODUÇÃO DE AMOXILINA**
### ***POR SÍNTESE ENZIMÁTICA EM BIORREATOR BATELADA E BATELADA ALIMENTADA***

In [7]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import random
import scipy
import glob
import os

### Importando dados

In [8]:
CIs = []
def convert_ci(ci):
    if type(ci) == str:
        ci_list = [float(i) for i in ci.split('_')]
        return ci_list
    else:
        ci_str = str(ci[0]) + '_' + str(ci[1]) + '_' + str(ci[2]) + '_' + str(ci[3])
        return ci_str
def import_dfs():
    # Path to the folder containing CSV files
    folder_path = '../data/'

    # Get a list of all CSV files in the folder
    csv_files = glob.glob(os.path.join(folder_path, '*.csv'))
    # Initialize an empty list to hold dataframes
    data_frames = {}

    # Read the first CSV file to establish the schema
    schema = ['ester_mm', 'amox_mm', 'apa_mm', 'aoh_mm','apa_t']
    all_times = []
    # Load each remaining CSV file, reorder columns, and append to the list
    for file in csv_files:
        df = pd.read_csv(file)  
        print(file)
        df = df[schema]  # Reorder columns to match the schema
        print(df['apa_t'].max())
        all_times.append(df['apa_t'].to_list())
        ci = df.iloc[0,:4].to_numpy()
        data_frames[convert_ci(ci)] = df
        CIs.append(convert_ci(ci))
    
    csv_files = [name.split('\\')[-1].rstrip('.csv') for name in csv_files]

    return data_frames,csv_files,all_times
data_frames,file_list,all_times = import_dfs()

print(data_frames.keys())
print(data_frames[CIs[0]].columns)
print(file_list)
print(CIs)


../data\25.45nh60ab20.csv
490.0
../data\25.46nh30ab30.csv
490.0
../data\25.47nh5ab80.csv
240.0
../data\5.100nh40ab80.csv
600.0
../data\5.102nh20ab40.csv
540.0
../data\5.103nh40ab30.csv
540.0
../data\5.106nh100ab40.csv
360.0
../data\5.107nh55ab55.csv
50.0
../data\5.109nh5ab55.csv
60.0
../data\5.110nh10ab55.csv
60.0
../data\5.112nh78ab35.csv
360.0
../data\5.48nh20ab80.csv
60.0
../data\5.80nh12ab40.csv
410.0
../data\5.81nh5ab30.csv
285.0
../data\nh60ab80.csv
600.0
dict_keys(['20.0_0.0_60.0_0.0', '30.0_0.0_30.0_0.0', '80.0_0.0_5.0_0.0', '80.0_0.0_40.0_0.0', '40.5_0.0_21.75_3.372620126926563', '30.4_0.0_43.0_2.55', '40.038986354775815_0.0640149834167829_100.04518072289156_1.4210854715202004', '55.0_0.0_55.0_0.623377', '55.0_0.0_5.0_0.0', '55.0_0.0_10.0_0.0', '32.5_0.0_78.0_2.5', '80.0_0.0_20.0_0.0', '38.75_0.0_12.0_1.4', '30.0_0.0_5.0_0.0', '80.0_0.0_60.0_0.0'])
Index(['ester_mm', 'amox_mm', 'apa_mm', 'aoh_mm', 'apa_t'], dtype='object')
['25.45nh60ab20', '25.46nh30ab30', '25.47nh5ab80', '5.

### Model

In [9]:
def enzymic_amox(t,y, 
k2,
k3,
k4,
KS,
KP,
KN,
kmenos4,
):
    Cez = 1
    CAB = y[0]
    CAN = y[1]
    CNH = y[2]
    CAOH = y[3]

    # fixed parameter
    k5 = 0.150

    RP = (Cez)/(k3 * KN + k4 * CNH + k5 * CNH) * (((k2 * k4 * CAB * CNH)/ (KS)) - ((kmenos4 * CAN * (k3 * KN + k5 * CNH))/KP))
    RB = (Cez * (k3 * KN + k5 * CNH) / (k3 * KN + k4 * CNH + k5 * CNH)) * (((k2 * CAB)/KS) - ((kmenos4 * CAN) / KP))
    RS = -(RP + RB)
    RNu = -(RP)

    dy = np.zeros(4)

    dy[0] =  RS    #  Ester  
    dy[1] =  RP    #  Amox
    dy[2] =  -(RP)    # 6-APA
    dy[3] = RB     #  AOh
    
    return np.array(dy)

# ODE Solver - ode15s similar

In [10]:
# ODE15s solver

def ode15s_amox(P, CI, t):
    try:
        sol = scipy.integrate.solve_ivp(
            enzymic_amox, 
            t_span=(t[0], t[-1]), 
            t_eval=t, 
            y0=CI, 
            method='BDF', 
            args=P, 
            #atol=1e-8, 
            #tol=1e-6
        )
        if sol.status != 0:
            raise ValueError("ODE solver failed to converge")
        return sol.y.T
    except Exception as e:
        print(f"Solver failed with error: {e}")
        # Handle solver failure (e.g., return NaNs or retry with different parameters)
        return np.full((len(CI), len(t)), np.nan).T

# Markov-Chain Monte-Carlo com Metropolis Hastings

In [11]:
def mcmc(df,P,name,N=5000,ode_solver=ode15s_amox,status=False,seed=42):
    if seed is not None:
        np.random.seed(seed)
        random.seed(seed)
    Np = len(P)
    # Extraindo dados
    CAB_medida  = df.loc[:,'ester_mm'].dropna().to_numpy()
    CAN_medida = df.loc[:,'amox_mm'].dropna().to_numpy()
    CNH_medida = df.loc[:,'apa_mm'].dropna().to_numpy()
    CAOH_medida = df.loc[:,'aoh_mm'].dropna().to_numpy()
    t = df.loc[:,'apa_t'].dropna().to_numpy()
    experiments = [CAB_medida,CAN_medida,CNH_medida,CAOH_medida]

    Cez = 1

    CI = np.array([CAB_medida[0],CAN_medida[0],CNH_medida[0],CAOH_medida[0]])

    Y = ode_solver(P,CI,t)
    CAB_ref   =  Y[:,0]      
    CAN_ref   =  Y[:,1]    
    CNH_ref   =  Y[:,2]      
    CAOH_ref  =  Y[:,3]
    desvio_CAB =  0.1*max(CAB_ref) 
    desvio_CAN =  0.1*max(CAN_ref)
    desvio_CNH =  0.1*max(CNH_ref)
    desvio_CAOH = 0.1*max(CAOH_ref)
    
    waux      = 6e-3 
    media_g   = 1       #Média gaussiana
    desviop_g = 0.6     #Desvio dos parâmetros ao utilizar priori gaussian

    estimate = [i for i in range(Np)]
    Nfix = len(estimate)

    p_ref = P                   
    p_old = P                     

    media_MCMC = media_g*P
    desvio_P   = desviop_g*P

    w = np.ones((1,Np))   
    w = w*waux  

    parametro_exato = (p_ref*np.ones((N,1))).T
    
    aceitacao    = np.zeros((1,N))  
    cadeia       = np.zeros((Np,N)) 
    conv_likeli  = np.zeros((1,N))
    k=0


    Lk_1 = np.dot((CAB_medida - CAB_ref),(CAB_medida - CAB_ref).T) / (desvio_CAB**2)     
    Lk_2 = np.dot((CAN_medida - CAN_ref),(CAN_medida - CAN_ref).T) / (desvio_CAN**2)      
    Lk_3 = np.dot((CNH_medida - CNH_ref),(CNH_medida - CNH_ref).T) / (desvio_CNH**2)       
    Lk_4 = np.dot((CAOH_medida - CAOH_ref),(CAOH_medida - CAOH_ref).T) / (desvio_CAOH**2)  
    
    lk_old = Lk_1 + Lk_2 + Lk_3 + Lk_4
    prior_old = np.sum((((p_old - media_MCMC))/((desvio_P)**2)))
    cadeia[:,0] = p_old
    
    # Contador para verificar a aceitação
    for i in range(0, N):  # Python index starts at 0, MATLAB at 1
        progress = (i / N) * 100
        if status and i % 100 == 0:  # Print progress every 100 iterations
            print(f'Progress: {progress:.3f}%', end='\r')
        # New parameter vector
        P_new = p_old + w * np.random.randn(Np) * p_old

        # Simulate using ODE solver
        Y = ode_solver(P_new[0], CI,t)

        CAB  = Y[:,0]
        CAN  = Y[:,1]        
        CNH  = Y[:,2]
        CAOH = Y[:,3]

        # Calculate new prior
        Prior_new = np.sum(((P_new - media_MCMC) / desvio_P) ** 2)

        # Likelihood calculations
        Lk_1 = np.dot((CAB_medida - CAB),(CAB_medida - CAB).T) / (desvio_CAB**2)        
        Lk_2 = np.dot((CAN_medida - CAN),(CAN_medida - CAN).T) / (desvio_CAN**2)         
        Lk_3 = np.dot((CNH_medida - CNH),(CNH_medida - CNH).T) / (desvio_CNH**2)        
        Lk_4 = np.dot((CAOH_medida - CAOH),(CAOH_medida - CAOH).T) / (desvio_CAOH**2)   
    
        Lk_new = Lk_1 + Lk_2 + Lk_3 + Lk_4
        
        # MCMC acceptance check
        if np.log(random.random()) < (-0.5 * (Lk_new + Prior_new - lk_old - prior_old)):
            p_old = P_new
            lk_old = Lk_new
            prior_old = Prior_new
            k += 1
        
        
        # Store results
        aceitacao[0,i] = k
        cadeia[:, i] = p_old
        conv_likeli[0,i] = lk_old + prior_old
    
    fig,ax = plt.subplots(2,4,figsize=[20,8])
    ax = ax.flatten()
   
    labels = ['k2',
    'k3',
    'k4',
    'KS',
    'KP',
    'KN',
    'kmenos4']


    for idx,g in enumerate(ax):
        if idx >= Np:
            continue
        g.plot(cadeia[idx,:])    
        g.set_title(labels[idx])
        g.set_ylabel("Value")
        g.set_xlabel("Iteration")
    fig.savefig(f"../results/model_donald/chains/{N}_single_{name}.png",dpi=400)
    plt.close()    

    aq = int(0.9 * N)  # Starting index for the burn-in period
    IC = 0.99          # Confidence interval level

    # Confidence interval bounds
    xaux = (1 - IC) / 2
    IC_inf = xaux
    IC_sup = 1 - xaux
    estimate = [i for i in range(Np)]

    amostra = N - aq + 1
    amostras = [] # [CIs, Nics, t, subs]


    output_par = np.zeros((amostra,Np))
    for i in range(len(CIs)):
        t = all_times[i]
        inst_amostra = np.zeros((len(list(range(aq, N + 1))),len(t),4))

        for idx,j in enumerate(range(aq, N + 1)):
            ii = j - aq
            paux = cadeia[:, j - 1]
            output_par[ii,:] = paux
            
            Y = ode_solver(paux,convert_ci(CIs[i]),t)
            inst_amostra[idx,:,0]  = Y[:,0]
            inst_amostra[idx,:,1] = Y[:,1]
            inst_amostra[idx,:,2] = Y[:,2]
            inst_amostra[idx,:,3] = Y[:,3]
        
        amostras.append(inst_amostra)


    AB_media = []
    AN_media = []
    NH_media = []
    AOH_media =[]
    AB_sup = []
    AN_sup = []
    NH_sup = []
    AOH_sup =[]
    AB_inf = []
    AN_inf = []
    NH_inf = []
    AOH_inf =  []   

    for i in range(len(CIs)):
        t = all_times[i]
        amostra = amostras[i]
        
        AB_mean = np.zeros(len(t))
        AB_inferior = np.zeros(len(t))
        AB_superior = np.zeros(len(t))
        NH_mean = np.zeros(len(t))
        NH_inferior = np.zeros(len(t))
        NH_superior = np.zeros(len(t))

        for j in range(len(t)):    
            AB_mean[j] = np.mean(amostra[:,j,0])
            y = np.percentile(amostra[:, j,0], [IC_inf * 100, IC_sup * 100])
            AB_inferior[j] = y[0]
            AB_superior[j] = y[1]

            NH_mean[j] = np.mean(amostra[:, j,2])
            y = np.percentile(amostra[:, j,2], [IC_inf * 100, IC_sup * 100])
            NH_inferior[j] = y[0]
            NH_superior[j] = y[1]
        
        AN_mean = np.zeros(len(t))
        AN_inferior = np.zeros(len(t))
        AN_superior = np.zeros(len(t))
        AOH_mean = np.zeros(len(t))
        AOH_inferior = np.zeros(len(t))
        AOH_superior = np.zeros(len(t))
        
        for j in range(len(t)):
            AN_mean[j] = np.mean(amostra[:, j, 1])
            y = np.percentile(amostra[:, j,1], [IC_inf * 100, IC_sup * 100])
            AN_inferior[j] = y[0]
            AN_superior[j] = y[1]

            AOH_mean[j] = np.mean(amostra[:, j,3])
            y = np.percentile(amostra[:, j,3], [IC_inf * 100, IC_sup * 100])
            AOH_inferior[j] = y[0]
            AOH_superior[j] = y[1]

        AB_media.append(AB_mean)
        NH_media.append(NH_mean)
        AB_inf.append(AB_inferior)
        AB_sup.append(AB_superior)
        NH_inf.append(NH_inferior)
        NH_sup.append(NH_superior)
        AN_media.append(AN_mean)
        AOH_media.append(AOH_mean)
        AN_inf.append(AN_inferior)
        AN_sup.append(AN_superior)
        AOH_inf.append(AOH_inferior)
        AOH_sup.append(AOH_superior)

    # individual plot
    # Ensure 'results' directory exists
    results_dir = f'../results/model_donald/'

    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
    for i in range(len(CIs)):
        names = ['POH-PGME','Amoxicillin','6-APA','POHPG']
        t = all_times[i]
        style = ['or','og','ob','ok']
        mstyle = ['-r','-g','-b','-k']
        medidas = [
            [AB_media[i], AB_inf[i], AB_sup[i]],
            [AN_media[i], AN_inf[i], AN_sup[i]],
            [NH_media[i], NH_inf[i], NH_sup[i]],
            [AOH_media[i],AOH_inf[i],AOH_sup[i]]
        ]
        fig,ax = plt.subplots(2,2,figsize=(10,10))
        for idx,axis in enumerate(ax.flatten()):
            sname = names[idx]
            medida = medidas[idx]
            
            axis.plot(t, data_frames[CIs[i]].iloc[:,idx].to_numpy(), style[idx], label=sname)
            axis.plot(t, medida[0], mstyle[idx])
            axis.plot(t, medida[1], '--r',linewidth=0.5)
            axis.plot(t, medida[2], '--r',linewidth=0.5) 
            axis.set_xlabel('Tempo (min)')
            axis.set_ylabel(f'Concentração de {sname} (mm)')  # Use f-string for proper string formatting
            axis.legend()  # Adjust legend position
        fig.savefig(os.path.join(results_dir, f"{name}_{N}_{file_list[i]}_mcmc.png"), dpi=300)
        plt.close()  # Close the figure to release memory
    return output_par,k/N

# Distribuição a priori dos parâmetros cinéticos

In [12]:
Cez = 1

k2 = 187 / 60
k3 = 44 / 60
k4 = 235 / 60
KS = 0.38 * 1000
KP = 0.095 * 1000
KN = 0.043 * 1000
kmenos4 = 217 / 60

P = np.zeros(7)
P[0]   = k2 
P[1]   = k3    
P[2]   = k4      
P[3]   = KS      
P[4]   = KP     
P[5]   = KN      
P[6]   = kmenos4 

Np = len(P)

labels = ['k2',
'k3',
'k4',
'KS',
'KP',
'KN',
'kmenos4']


In [13]:
priori = P
priori_name = 'donald'

In [14]:
print(file_list)
print(CIs)

['25.45nh60ab20', '25.46nh30ab30', '25.47nh5ab80', '5.100nh40ab80', '5.102nh20ab40', '5.103nh40ab30', '5.106nh100ab40', '5.107nh55ab55', '5.109nh5ab55', '5.110nh10ab55', '5.112nh78ab35', '5.48nh20ab80', '5.80nh12ab40', '5.81nh5ab30', 'nh60ab80']
['20.0_0.0_60.0_0.0', '30.0_0.0_30.0_0.0', '80.0_0.0_5.0_0.0', '80.0_0.0_40.0_0.0', '40.5_0.0_21.75_3.372620126926563', '30.4_0.0_43.0_2.55', '40.038986354775815_0.0640149834167829_100.04518072289156_1.4210854715202004', '55.0_0.0_55.0_0.623377', '55.0_0.0_5.0_0.0', '55.0_0.0_10.0_0.0', '32.5_0.0_78.0_2.5', '80.0_0.0_20.0_0.0', '38.75_0.0_12.0_1.4', '30.0_0.0_5.0_0.0', '80.0_0.0_60.0_0.0']


In [15]:
print(data_frames[CIs[3]])
name_teste = 'teste'
ps,aceita = mcmc(data_frames[CIs[3]],P=priori,name=name_teste,N=10,status=True,seed=10)

     ester_mm    amox_mm     apa_mm     aoh_mm  apa_t
0   80.000000   0.000000  40.000000   0.000000    0.0
1   78.905047   0.565504  39.547597   0.342173    5.0
2   77.702135   0.898980  38.491989   0.772553   15.0
3   78.494440   0.904807  39.095193   0.684346   20.0
4   76.167665   2.035815  37.822473   1.710864   25.0
5   73.175223   4.421924  35.594902   2.576352   60.0
6   64.597898   8.192303  31.736180   7.224603  120.0
7   60.074144  10.467944  29.437747   9.721766  180.0
8   52.782365  11.322744  28.609807  15.781445  240.0
9   49.307620  12.527604  27.380142  18.146150  300.0
10  43.987574  12.486554  27.454955  23.648201  360.0
11  43.130362  10.068250  29.840648  26.739491  420.0
12  37.729138  11.831538  28.036082  30.423476  480.0
13  35.065351  10.837702  29.112040  33.776951  540.0
14  33.802063   7.406971  32.546002  38.536327  600.0


In [16]:
import os

Ns = [50000]
num_of_replicates = 3
seed = 0

# Define the log file path
log_file_path = "../results/model_donald/aceitacao_log_adj.txt"

# Ensure the log file exists by creating it if it doesn't exist
if not os.path.exists(log_file_path):
    with open(log_file_path, "w") as f:
        pass

with open(log_file_path, "a") as log_file:
    for curr in range(num_of_replicates):
        for N in Ns:
            all_par_df = pd.DataFrame(np.zeros((len(data_frames),len(labels))),columns=labels)
            all_parstderror_df = pd.DataFrame(np.zeros((len(data_frames),len(labels))),columns=labels)
            count = 0
            for ci,name in zip(CIs,file_list):
                
                print(f'Currently {curr}: {name} {N}')
                Ps,aceitacao = mcmc(data_frames[ci],P=priori,name=f"{curr}_{name}_adj",N=N,status=True,seed=seed)
                np.savez_compressed(f'../results/pickles/donald/{curr}_{N}_{priori_name}_{name}_adj',Ps)
                
                # Log the aceitacao variable
                log_entry = f"{curr}_{N}_{priori_name}_{name}_adj : {aceitacao}\n"
                log_file.write(log_entry)  # Append the formatted entry to the log file
                print('\naceitação: ', aceitacao)
            
                    
                P_conf = pd.DataFrame(Ps)
                P_mean = P_conf.describe().iloc[1,:].to_numpy()
                P_error = P_conf.describe().iloc[2,:].to_numpy() / np.sqrt(P_conf.size)

                for i in range(len(labels)):
                    print(f'{labels[i]}: {P_mean[i]} +- {P_error[i]}')
                
                all_par_df.iloc[count,:] = P_mean
                all_parstderror_df.iloc[count,:] = P_error
                count += 1
                seed +=1

            all_par_df.to_csv(f'../results/model_donald/par_adj/{curr}_{N}_{priori_name}_mean.csv')
            all_parstderror_df.to_csv(f'../results/model_donald/par_adj/{curr}_{N}_{priori_name}_error.csv')

Currently 0: 25.45nh60ab20 50000
Progress: 99.800%
aceitação:  0.5373
k2: 0.2684481315597963 +- 0.00010083140761463954
k3: 0.8349876293960727 +- 0.00039761627926709713
k4: 0.022927586634837138 +- 1.4018097322549762e-05
KS: 528.3195212928396 +- 0.19787145553496885
KP: 165.16937035768163 +- 0.13711805002259994
KN: 53.25704759272599 +- 0.02555278061586729
kmenos4: 0.08175988937309912 +- 8.975941587917358e-05
Currently 0: 25.46nh30ab30 50000
Progress: 99.800%
aceitação:  0.41326
k2: 1.023985749689344 +- 0.0007421326816722571
k3: 1.0369465871367798 +- 0.0005694688270933965
k4: 0.6955508127729719 +- 0.00044674138126191105
KS: 435.1846029981388 +- 0.39151637409995893
KP: 34.83809587983242 +- 0.011392000998472231
KN: 63.1530435755168 +- 0.026801079948831477
kmenos4: 6.3201218038698315 +- 0.005071777753079844
Currently 0: 25.47nh5ab80 50000
Progress: 99.800%
aceitação:  0.9297
k2: 0.29421542083383234 +- 0.00019507474023930442
k3: 0.9287238925813078 +- 0.0004398345828706015
k4: 1.057458900275123

c:\Users\lneoc\Documents\art\bioreactor_mcmc\venv\Lib\site-packages\scipy\integrate\_ivp\bdf.py:417: RuntimeWarning: invalid value encountered in subtract
  D[order + 2] = d - D[order + 1]


Progress: 99.800%
aceitação:  0.49152
k2: 0.6336221731844646 +- 0.000305609020443833
k3: 0.4161837183355232 +- 0.00040880330245006815
k4: 0.11597179511293808 +- 3.58630700066649e-05
KS: 378.4989700508391 +- 0.12230219434859586
KP: 67.20743756986397 +- 0.025637464788576907
KN: 54.02385577670199 +- 0.020957798653849898
kmenos4: 5.279401450273665 +- 0.001930285458176456
Currently 2: 25.47nh5ab80 50000


c:\Users\lneoc\Documents\art\bioreactor_mcmc\venv\Lib\site-packages\scipy\integrate\_ivp\bdf.py:417: RuntimeWarning: invalid value encountered in subtract
  D[order + 2] = d - D[order + 1]


Progress: 99.800%
aceitação:  0.93232
k2: 0.6559627276962805 +- 0.00030464245762364133
k3: 0.9562184394452635 +- 0.0005953923197385188
k4: 1.3100935558443454 +- 0.0003832499806375261
KS: 111.04136329868551 +- 0.040106219384674284
KP: 115.13954865383243 +- 0.11455996160972633
KN: 30.144881294102504 +- 0.016981903676794172
kmenos4: 2.010042500388567 +- 0.0028159942808638604
Currently 2: 5.100nh40ab80 50000
Progress: 99.800%
aceitação:  0.23196
k2: 0.8024988478799373 +- 0.0001620082623956092
k3: 0.5961293554256067 +- 0.00023684698443478578
k4: 0.08747246157512024 +- 3.151703099851957e-05
KS: 667.9609970563449 +- 0.12303084595005041
KP: 30.47047604257382 +- 0.007804508400262695
KN: 46.14694227378486 +- 0.01808842450325922
kmenos4: 5.233757303961045 +- 0.0008641793662263259
Currently 2: 5.102nh20ab40 50000
Progress: 99.800%
aceitação:  0.7742
k2: 1.0462931628338321 +- 0.00036584440028013607
k3: 0.7480598239842048 +- 0.0010296658390886392
k4: 0.3728319712072256 +- 0.000171643653472224
KS: 19

In [17]:
Ns = [100000]
num_of_replicates = 3
curr = 0
seed = 30

# Define the log file path
log_file_path = "../results/model_donald/aceitacao_log_adj.txt"

# Ensure the log file exists by creating it if it doesn't exist
if not os.path.exists(log_file_path):
    with open(log_file_path, "w") as f:
        pass

with open(log_file_path, "a") as log_file:
    for curr in range(num_of_replicates):
        for N in Ns:
            all_par_df = pd.DataFrame(np.zeros((len(data_frames),len(labels))),columns=labels)
            all_parstderror_df = pd.DataFrame(np.zeros((len(data_frames),len(labels))),columns=labels)
            count = 0
            for ci,name in zip(CIs,file_list):
                
                print(f'Currently {curr}: {name} {N}')
                Ps,aceitacao = mcmc(data_frames[ci],P=priori,name=f"{curr}_{name}_adj",N=N,status=True,seed=seed)
                np.savez_compressed(f'../results/pickles/donald/{curr}_{N}_{priori_name}_{name}',Ps)
                print('\naceitação: ',aceitacao)
            
                    
                P_conf = pd.DataFrame(Ps)
                P_mean = P_conf.describe().iloc[1,:].to_numpy()
                P_error = P_conf.describe().iloc[2,:].to_numpy() / np.sqrt(P_conf.size)

                for i in range(len(labels)):
                    print(f'{labels[i]}: {P_mean[i]} +- {P_error[i]}')
                
                all_par_df.iloc[count,:] = P_mean
                all_parstderror_df.iloc[count,:] = P_error
                count += 1
                seed +=1

            all_par_df.to_csv(f'../results/model_donald/{curr}_{N}_{priori_name}_mean.csv')
            all_parstderror_df.to_csv(f'../results/model_donald/{curr}_{N}_{priori_name}_error.csv')

Currently 0: 25.45nh60ab20 100000
Progress: 99.900%
aceitação:  0.54156
k2: 0.14184928714148462 +- 5.8209285734829146e-05
k3: 0.6423130400631031 +- 0.0006189533348332783
k4: 0.0010885822313610864 +- 8.465854442443546e-07
KS: 286.1308144981343 +- 0.11475195135836161
KP: 23.18084891556102 +- 0.0285155976647731
KN: 19.17543813724596 +- 0.0062847920958945
kmenos4: 0.13441100033462886 +- 0.00012184879835501617
Currently 0: 25.46nh30ab30 100000
Progress: 99.900%
aceitação:  0.48858
k2: 0.7354540673508361 +- 0.00033704955454786457
k3: 0.662855416591613 +- 0.0003952653373732468
k4: 0.11152076132144217 +- 5.4697666910518036e-05
KS: 457.25379226499007 +- 0.1493513071419246
KP: 14.320336836240784 +- 0.0035582768293939415
KN: 40.45207937331689 +- 0.02792594225856259
kmenos4: 1.7530081317098494 +- 0.0013942310584978033
Currently 0: 25.47nh5ab80 100000
Progress: 99.900%
aceitação:  0.93246
k2: 0.0982250257259518 +- 8.096350028639009e-05
k3: 0.946669025235718 +- 0.0009629597355190519
k4: 1.3512032127

c:\Users\lneoc\Documents\art\bioreactor_mcmc\venv\Lib\site-packages\scipy\integrate\_ivp\bdf.py:417: RuntimeWarning: invalid value encountered in subtract
  D[order + 2] = d - D[order + 1]


Progress: 99.900%
aceitação:  0.9334
k2: 4.2600555851673985 +- 0.003499025324793926
k3: 0.15406377384088576 +- 0.000157079925983196
k4: 0.029470768791905674 +- 1.576336987126881e-05
KS: 654.3349861709878 +- 0.4116568897820615
KP: 43.05612286845223 +- 0.02560044468368703
KN: 0.902418625307716 +- 0.0006067945482714478
kmenos4: 1.1132267371451887 +- 0.0013486599718359404
Currently 1: nh60ab80 100000
Progress: 99.900%
aceitação:  0.05722
k2: 0.6331146169382553 +- 4.467773527712556e-05
k3: 1.8684558482132583 +- 0.00016809011448310233
k4: 0.02123741934326094 +- 4.153978696887095e-06
KS: 790.882917657258 +- 0.05545308218567246
KP: 121.60707787317067 +- 0.0317300561076877
KN: 114.53256889254666 +- 0.025936838795890576
kmenos4: 1.9205975338866235 +- 0.0004914636233692566
Currently 2: 25.45nh60ab20 100000
Progress: 99.900%
aceitação:  0.34862
k2: 0.3180144779748993 +- 8.15045327822836e-05
k3: 0.18188753468200652 +- 9.631548442043384e-05
k4: 0.16769553317791805 +- 2.038916045646216e-05
KS: 70.347

In [18]:
seed = 43